In [25]:
import os
import sys
sys.path.append('./')
sys.path.append('../')
import numpy as np

from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

from art.attacks.evasion import FastGradientMethod
from art.estimators.classification import SklearnClassifier
from sklearn.datasets import load_wine

from Core.dataset import CDataset

# Step 1: Load dataset

## Wine Dataset

In [ ]:
data = load_wine()
X, y = data.data, data.target

# Standardize features for stability
scaler = StandardScaler()
X = scaler.fit_transform(X)

print(np.unique(y, return_counts=True))


## Kyoto

In [26]:
cDATA = 'Kyoto'

objDS = CDataset()

strFileName =  r'../../DATA/Kyoto2015DS.csv'
#strPath = os.path.join(strRoot, strFileName)
X, y = objDS.GetKyotoDataset(strFileName)
print('Kyoto:', X.shape, y.shape)

fTestSize = 0.9

CDataset Object Created
label
0    30000
1    30000
dtype: int64
Kyoto: (60000, 13) (60000, 1)


## Spilt Data

In [27]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=fTestSize, 
                                                    random_state=42)

In [ ]:
print('Training Data:', X_train.shape, y_train.shape)
print('Testing Data:', X_test.shape, y_test.shape)

print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

## One Hot encoding

In [28]:
enc = OneHotEncoder()
# Training
enc.fit(y_train)  
y_train_cat = enc.transform(y_train).toarray()

# Training
enc.fit(y_test)  
y_test_cat = enc.transform(y_test).toarray()

#print(y_train[0:5])
#print(y_test_cat[0:5])

# Step 2: Train a simple classifier

In [30]:
# model = RandomForestClassifier() not working
#model = SVC(C=1.0, kernel="rbf")
model = SVC(C=1.0, kernel="rbf")
model.fit(X_train, y_train)

# Evaluate on clean test set
acc_clean = model.score(X_test, y_test)
print(f"Accuracy on clean test set: {acc_clean:.4f}")

c:\Users\admin\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\utils\validation.py:1111: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Accuracy on clean test set: 0.6480


# Step 3: Wrap model for ART

In [31]:
classifier = SklearnClassifier(model=model)

classifier.fit(X_train, y_train_cat)

predictions = classifier.predict(X_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test_cat, axis=1)) / len(y_test_cat)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 64.79814814814814%


# Step 4: Create adversarial attack (FGSM)

In [32]:
attack = FastGradientMethod(estimator=classifier, eps=0.1)

# Step 5: Generate adversarial examples

In [ ]:
X_test_adv = attack.generate(x=X_test)

# Step 6: Evaluate on adversarial set

In [ ]:
acc_adv = model.score(X_test_adv, y_test)
print(f"Accuracy on adversarial test set: {acc_adv:.4f}")

In [ ]:
# Optional: Compare a few original vs adversarial samples
print("\nOriginal sample (first row):", X_test[0])
print("Adversarial sample (first row):", X_test_adv[0])